# <center>载入数据</center>

In [58]:
import pandas as pd
import numpy as np

data = pd.read_csv("f:/机器学习/kaggle/train.csv")
#print(data['target'].replace('Class_8',8))
data['target'].replace('Class_9',int(9),inplace=True)
data['target'].replace('Class_8',int(8),inplace=True)
data['target'].replace('Class_7',int(7),inplace=True)
data['target'].replace('Class_6',int(6),inplace=True)
data['target'].replace('Class_5',int(5),inplace=True)
data['target'].replace('Class_4',int(4),inplace=True)
data['target'].replace('Class_3',int(3),inplace=True)
data['target'].replace('Class_2',int(2),inplace=True)
data['target'].replace('Class_1',int(1),inplace=True)
#data['target'].to_numeric()
data['target'].astype(int)
data_0 = pd.read_csv("f:/机器学习/kaggle/test.csv")


# <center>数据预处理</center>

## 无量纲化 - 标准化StandardScaler

In [20]:
#无量纲化 - 标准化
from sklearn.preprocessing import MinMaxScaler
#print(StandardScaler().fit_transform(data.loc[:,'feature_0':'feature_74']))
data.loc[:,'feature_0':'feature_74'] = MinMaxScaler().fit_transform(data.loc[:,'feature_0':'feature_74'])


In [46]:
from sklearn.preprocessing import StandardScaler
#print(StandardScaler().fit_transform(data.loc[:,'feature_0':'feature_74']))
data.loc[:,'feature_0':'feature_74'] = StandardScaler().fit_transform(data.loc[:,'feature_0':'feature_74'])

# <center>降维处理</center>

In [21]:
#from sklearn.decomposition import PCA
#pca = PCA(n_components=27)


In [56]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=7)
X_train = data.loc[:,'id':'feature_74']
y_train = data['target']
lda.fit(X_train,y_train)
X_train = lda.transform(X_train)
X_test = data_0.loc[:,'id':'feature_74']
#lda(n_components=7).fit_transform(X_train,y_train)
print(X_train)
print(X_train.shape)

[[ 1.15270822 -2.19585473  0.6094214  ...  0.16553743  0.99157792
  -0.44286796]
 [-0.89823504 -0.85062366 -0.11871815 ...  0.15539811  0.17332528
   0.27200049]
 [-0.46818562 -0.05420531 -0.51169966 ... -0.03833684  0.47088714
   0.09435096]
 ...
 [ 1.07951603  0.27845118  2.82748996 ...  2.19616487  0.03284492
   0.40961607]
 [-0.74005371 -0.4865265  -0.16367293 ... -0.6375635  -0.51860837
   0.04306263]
 [ 0.78447383 -0.03474731 -0.38198373 ... -2.33569752 -3.58955449
   2.23480402]]
(200000, 7)


In [44]:
X_train = data.loc[:,'id':'feature_74']
y_train = data['target']
#X_train = np.array(pca.fit_transform(X_train))
X_test = data_0.loc[:,'id':'feature_74']
X_train = pca.fit_transform(X_train.drop(columns='id'))
#print(X_train)
#y_train = np.array(pd.get_dummies(y_train))
#print(X_train)
print(X_train.shape)

(200000, 27)


# <center>第一个模型：DF21</center>

In [57]:
from deepforest import CascadeForestClassifier
import xgboost
model = CascadeForestClassifier(n_jobs=-1,n_estimators=3,n_trees=200,max_depth=12,min_samples_leaf=5)
model.fit(X_train, y_train.values)
#.drop(columns='id').values


[2021-06-30 17:56:04.721] Start to fit the model:
[2021-06-30 17:56:04.722] Fitting cascade layer = 0 
[2021-06-30 17:56:49.673] layer = 0  | Val Acc = 34.725 % | Elapsed = 44.951 s
[2021-06-30 17:56:52.081] Fitting cascade layer = 1 


KeyboardInterrupt: 

In [51]:
y_pred = model.predict_proba(X_test.drop(columns='id').values)
print(y_pred)

[2021-06-30 17:39:02.338] Start to evalute the model:
[2021-06-30 17:39:02.533] Evaluating cascade layer = 0 
[2021-06-30 17:39:08.454] Evaluating cascade layer = 1 
[2021-06-30 17:39:15.166] Evaluating cascade layer = 2 
[[0.03241831 0.01552918 0.01363079 ... 0.08837403 0.44762948 0.07355791]
 [0.04949125 0.03492403 0.03017911 ... 0.11277651 0.39063981 0.10431176]
 [0.04508268 0.0309021  0.02665711 ... 0.10677069 0.40151731 0.1005949 ]
 ...
 [0.03164271 0.01496197 0.01361309 ... 0.08706772 0.45339308 0.0732037 ]
 [0.053379   0.0416237  0.03651605 ... 0.10962274 0.3717686  0.11242556]
 [0.04954008 0.03550004 0.03049993 ... 0.11100419 0.39118595 0.10495191]]


In [52]:
out={}
out['id']=X_test['id']
df=pd.DataFrame(out)
df2=pd.DataFrame(y_pred,columns=['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])
#print(df2)
result=pd.concat([df,df2],axis = 1)
print(result)

           id   Class_1   Class_2   Class_3   Class_4   Class_5   Class_6  \
0      200000  0.032418  0.015529  0.013631  0.010327  0.007936  0.310597   
1      200001  0.049491  0.034924  0.030179  0.023217  0.017196  0.237265   
2      200002  0.045083  0.030902  0.026657  0.019178  0.014172  0.255125   
3      200003  0.051150  0.040708  0.034225  0.023814  0.020422  0.210330   
4      200004  0.049142  0.032104  0.028636  0.019658  0.014985  0.249008   
...       ...       ...       ...       ...       ...       ...       ...   
99995  299995  0.036632  0.021581  0.018037  0.013943  0.009106  0.284440   
99996  299996  0.036259  0.021672  0.017097  0.013305  0.009195  0.286139   
99997  299997  0.031643  0.014962  0.013613  0.009839  0.007949  0.308330   
99998  299998  0.053379  0.041624  0.036516  0.024284  0.018748  0.231633   
99999  299999  0.049540  0.035500  0.030500  0.021394  0.016320  0.239604   

        Class_7   Class_8   Class_9  
0      0.088374  0.447629  0.073558  

In [35]:

result.to_csv("f:/机器学习/kaggle/submission_0.csv",index=False)

# <center>第二个模型：随机森林模型</center>

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:

model_rf = RandomForestClassifier(n_jobs=2, n_estimators=100)
#model_rf.fit(X_train.drop(columns='id').values, y_train.values)
model_rf.fit(X_train, y_train.values)

RandomForestClassifier(n_jobs=2)

In [49]:
y_pred_rf = model_rf.predict_proba(pca.transform(X_test.drop(columns='id').values))

In [50]:
out_rf={}
out_rf['id']=X_test['id']
df_rf=pd.DataFrame(out_rf)
df2_rf=pd.DataFrame(y_pred_rf,columns=['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])
#print(df2)
result_rf=pd.concat([df_rf,df2_rf],axis = 1)
print(result_rf)

           id  Class_1   Class_2   Class_3  Class_4  Class_5   Class_6  \
0      200000     0.06  0.371667  0.235667     0.00     0.00  0.226667   
1      200001     0.03  0.090000  0.060000     0.02     0.01  0.300000   
2      200002     0.02  0.020000  0.000000     0.03     0.01  0.760000   
3      200003     0.03  0.160000  0.070000     0.03     0.00  0.310000   
4      200004     0.04  0.150000  0.060000     0.03     0.02  0.350000   
...       ...      ...       ...       ...      ...      ...       ...   
99995  299995     0.09  0.310000  0.200000     0.02     0.01  0.110000   
99996  299996     0.05  0.250000  0.090000     0.02     0.02  0.190000   
99997  299997     0.08  0.270000  0.180000     0.07     0.00  0.130000   
99998  299998     0.03  0.050000  0.050000     0.02     0.02  0.290000   
99999  299999     0.02  0.150000  0.040000     0.01     0.01  0.320000   

       Class_7  Class_8  Class_9  
0         0.01     0.02    0.076  
1         0.05     0.28    0.160  
2     

In [51]:
result_rf.to_csv("f:/机器学习/kaggle/submission_rf.csv",index=False)

## <center> 单独提交的成绩比DF21差很多</center>
![randomforest](f:/机器学习/kaggle/rfalone.jpg)

# <center>第三个模型：XGBOOST</center>

## 调包

In [5]:
from xgboost import XGBClassifier

## 训练模型

In [52]:
#X_train.drop(columns='id').values, y_train.values X_test.drop(columns='id').values
model_xgb = XGBClassifier(eval_metric='mlogloss')
#print(y_train.values-1)
model_xgb.fit(X_train,y_train.values)
y_pred_xgb = model_xgb.predict_proba(pca.transform(X_test.drop(columns='id').values))
print(y_pred_xgb)

C:\Users\王诗睿\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[[0.05675288 0.4249337  0.15678912 ... 0.01542367 0.04156385 0.12969825]
 [0.03152695 0.09978833 0.07856461 ... 0.0654236  0.27513596 0.1231351 ]
 [0.01384554 0.01046704 0.01290232 ... 0.02183015 0.10686731 0.03908109]
 ...
 [0.05284063 0.3240927  0.15186003 ... 0.02630306 0.06373379 0.12993288]
 [0.03748401 0.12308677 0.05059336 ... 0.06204379 0.35937983 0.17037001]
 [0.03493327 0.06069035 0.0780729  ... 0.06188824 0.28863567 0.14841767]]


## 结果

In [53]:
out_xgb={}
out_xgb['id'] = X_test['id']
df_xgb = pd.DataFrame(out_xgb)
df2_xgb = pd.DataFrame(y_pred_xgb,columns=['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])
#print(df2)
result_xgb=pd.concat([df_xgb,df2_xgb],axis = 1)
print(result_xgb)

           id   Class_1   Class_2   Class_3   Class_4   Class_5   Class_6  \
0      200000  0.056753  0.424934  0.156789  0.022784  0.009327  0.142727   
1      200001  0.031527  0.099788  0.078565  0.016905  0.017504  0.292017   
2      200002  0.013846  0.010467  0.012902  0.004138  0.007957  0.782912   
3      200003  0.042245  0.101807  0.062264  0.030384  0.025927  0.250909   
4      200004  0.029756  0.097880  0.104765  0.033432  0.008534  0.366349   
...       ...       ...       ...       ...       ...       ...       ...   
99995  299995  0.096795  0.320943  0.141071  0.032389  0.012533  0.096939   
99996  299996  0.051672  0.219423  0.142223  0.033840  0.017350  0.201759   
99997  299997  0.052841  0.324093  0.151860  0.037224  0.010551  0.203462   
99998  299998  0.037484  0.123087  0.050593  0.024847  0.014452  0.157743   
99999  299999  0.034933  0.060690  0.078073  0.020614  0.012782  0.293967   

        Class_7   Class_8   Class_9  
0      0.015424  0.041564  0.129698  

## 输出为csv

In [54]:
result_xgb.to_csv("f:/机器学习/kaggle/submission_xgb.csv",index=False)

## 单个模型提交结果
![xgboost](F:\机器学习\竞赛作业\xgb.jpg)

# <center>集成学习</center>
## smart ensembling

In [55]:
sub1 = pd.read_csv('f:/机器学习/kaggle/submission.csv')
sub2 = pd.read_csv('f:/机器学习/kaggle/submission_rf.csv')
sub3 = pd.read_csv('f:/机器学习/kaggle/submission_xgb.csv')

In [56]:
def generation(main, support, coeff): 
    sub1  = support.copy()
    sub1v = sub1.values    
    
    sub2  = main.copy() 
    sub2v = sub2.values
       
    imp  = main.copy()    
    impv = imp.values  
    NCLASS = 9
    number = 0
    
    for i in range (len(main)):               
        
        row1 = sub1v[i,1:]
        row2 = sub2v[i,1:]
        row1_sort = np.sort(row1)
        row2_sort = np.sort(row2) 
        
        row = (row2 * coeff) + (row1 * (1.0 - coeff))
        row_sort = np.sort(row)        
        
        for j in range (NCLASS):             
            if ((row2[j] == row2_sort[8]) and (row1[j] != row1_sort[8])):                                
                row = row2 
                number = number + 1            
#               print(number, i)
        
        impv[i, 1:] = row
    
    imp.iloc[:, 1:] = impv[:, 1:]
    print(f'Number of rows unchanged: {number}')
    print(30 * '=')                                        
    return imp

In [57]:
sub = generation(sub2, sub1, 0.4)
sub_ens = sub
display(sub_ens, sub_ens.describe().transpose())
sub_ens.to_csv('f:/机器学习/kaggle/sub.csv',index=False)

Number of rows unchanged: 23446


,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.0765,0.363167,0.188767,0.0060,0.0105,0.227167,0.0145,0.0155,0.0979
1,200001,0.0300,0.090000,0.060000,0.0200,0.0100,0.300000,0.0500,0.2800,0.1600
2,200002,0.0170,0.033500,0.003000,0.0165,0.0055,0.766000,0.0185,0.1025,0.0375
3,200003,0.0300,0.124000,0.068500,0.0375,0.0150,0.329500,0.0390,0.2410,0.1155
4,200004,0.0385,0.111000,0.067500,0.0315,0.0185,0.339500,0.0725,0.1835,0.1375
...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.0600,0.329500,0.179000,0.0335,0.0160,0.128000,0.0315,0.0910,0.1315
99996,299996,0.0500,0.262000,0.108000,0.0230,0.0215,0.170500,0.0475,0.1840,0.1335
99997,299997,0.0485,0.294000,0.187500,0.0475,0.0015,0.151000,0.0285,0.0965,0.1450
99998,299998,0.0390,0.059000,0.081500,0.0215,0.0170,0.272000,0.0795,0.3225,0.1080


,count,mean,std,min,25%,50%,75%,max
id,100000.0,249999.500000,28867.657797,200000.0,224999.7500,249999.5000,274999.2500,299999.000000
Class_1,100000.0,0.047392,0.020317,0.0,0.0365,0.0465,0.0575,0.775333
Class_2,100000.0,0.121580,0.100982,0.0,0.0580,0.0825,0.1500,0.841000
Class_3,100000.0,0.076118,0.045336,0.0,0.0445,0.0630,0.1000,0.770000
Class_4,100000.0,0.024493,0.012905,0.0,0.0170,0.0230,0.0305,0.600000
Class_5,100000.0,0.015994,0.010038,0.0,0.0100,0.0150,0.0205,0.620000
Class_6,100000.0,0.257118,0.133970,0.0,0.1800,0.2340,0.3000,0.997000
Class_7,100000.0,0.075218,0.030192,0.0,0.0545,0.0770,0.0960,0.640000
Class_8,100000.0,0.253052,0.093955,0.0,0.1900,0.2710,0.3280,0.620000
Class_9,100000.0,0.129034,0.038373,0.0,0.1080,0.1300,0.1500,0.692500


In [37]:
#resemble_0 = 0.1 * result.loc[:,'Class_1':'Class_9'] + 0.1 * result_rf.loc[:,'Class_1':'Class_9'] + 0.8 * result_xgb.loc[:,'Class_1':'Class_9'] 
#print(resemble_0)
#resemble_1={}
#resemble_1['id']=X_test['id']
#resemble_2=pd.DataFrame(resemble_1)
#resemble=pd.concat([resemble_2,resemble_0],axis=1)
#print(resemble)

           id   Class_1   Class_2   Class_3   Class_4   Class_5   Class_6  \
0      200000  0.047034  0.411369  0.162026  0.022776  0.012034  0.171408   
1      200001  0.033635  0.080075  0.079869  0.017739  0.015376  0.287785   
2      200002  0.029127  0.035474  0.022258  0.014908  0.005350  0.684857   
3      200003  0.043545  0.101202  0.099313  0.040998  0.025004  0.217595   
4      200004  0.036638  0.092919  0.080417  0.029078  0.015966  0.299173   
...       ...       ...       ...       ...       ...       ...       ...   
99995  299995  0.067131  0.400316  0.132764  0.023619  0.015677  0.104361   
99996  299996  0.055936  0.259947  0.115101  0.024950  0.015372  0.190515   
99997  299997  0.089366  0.234118  0.139148  0.028179  0.011982  0.157490   
99998  299998  0.031550  0.023405  0.020053  0.010994  0.013503  0.389009   
99999  299999  0.048186  0.075668  0.081103  0.028527  0.024223  0.290706   

        Class_7   Class_8   Class_9  
0      0.019530  0.046037  0.107786  

In [16]:
resemble.to_csv("f:/机器学习/kaggle/submission_rem.csv",index=False)

In [77]:
blend = sub1.copy()
columns = sub1.columns[1:]
blend[columns] = 0.95*0.6*sub1[columns] + 0.95*0.4*sub3[columns]+0.05*sub2[columns]
blend.to_csv('f:/机器学习/kaggle/sub_blend.csv', index=False)
blend.head()


,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.074441,0.383833,0.161138,0.014358,0.013519,0.195245,0.016336,0.023919,0.117210
1,200001,0.037705,0.085170,0.089855,0.021674,0.015701,0.269891,0.070111,0.269602,0.140291
2,200002,0.014811,0.029202,0.007753,0.007347,0.004949,0.774406,0.019270,0.101685,0.040576
3,200003,0.034653,0.103687,0.065635,0.037271,0.024102,0.306070,0.088793,0.209142,0.130646
4,200004,0.034682,0.093144,0.084136,0.032729,0.014218,0.346238,0.064690,0.196676,0.133488
